This is the code to create the figure used in the paper which shows the whole USA.

In [1]:
%matplotlib inline

# Assuming we are in the notebooks directory, we need to move one up:
%cd ..

from __future__ import print_function

import os
import us
import glob

import numpy as np
import pandas as pd
import seaborn as sns
import geopandas as gpd
import matplotlib.pyplot as plt

/home/olivia/GitHub/redistrict


In [2]:
states = gpd.read_file('./fig/geojson/state_outlines.geo.json')
# states = gpd.read_file('./fig/shapefile/state_outlines.shp')
delete_states = []
for index, row in states.iterrows():
#     print(index, row)
    st = us.states.lookup(row['STATEFP'])
    if st.is_territory:
        print(st, index)
        delete_states.append(index)
states = states.drop(states.index[delete_states])
def get_abbr(row):
#     print(row)
    st = us.states.lookup(row['STATEFP'])
#     state_name.append(str(st.abbr))
#     print(str(st.abbr))
    return str(st.abbr)
states['State'] = states.apply(get_abbr, axis =1)
improvement = pd.read_csv('./fig/csv/improvement.csv')
improvement = improvement[['State', 'Ratio of Mean Pairwise Distance of Clusters versus of Congressional Districts']]
improvement.rename(columns={'Ratio of Mean Pairwise Distance of Clusters versus of Congressional Districts': 'Ratio'}, inplace=True)
result = gpd.GeoDataFrame(pd.merge(improvement, states, on='State'))
result['Colour'] = 1 - result['Ratio']

American Samoa 51
Guam 52
Northern Mariana Islands 53
Puerto Rico 54
Virgin Islands 55


In [ ]:
# states.crs.pop('no_defs', None)
# result.crs.pop('no_defs', None)
result.crs = {'init': u'epsg:4326'}
states.crs = {'init': u'epsg:4326'}
# result.crs = None
# states.crs = None

print(result.crs, states.crs)

In [ ]:
fig, ax = plt.subplots()
ax.set_aspect('equal')
# states = states.to_crs({'init': 'epsg:3395'})
states.plot(ax=ax, color='#888888', linewidth=0.1)
# result = result.to_crs({'init': 'epsg:3395'})
result.plot(ax=ax, linewidth=0.1, column='Colour', cmap='OrRd')
plt.axis('off')
# plt.savefig('./fig/usa.pdf', bbox_inches='tight')
plt.show();

In [ ]:
states.to_file('./fig/shapefile/results/states.shp', driver="ESRI Shapefile")
result.to_file('./fig/shapefile/results/result.shp', driver="ESRI Shapefile")

In [ ]:
usa = gpd.read_file('./fig/shapefile/States/States.shp')


In [ ]:
usa.plot()

In [ ]:
usa.head()

In [ ]:
del result['geometry']

In [ ]:
result.head()

In [ ]:
merged = pd.merge(usa, result, left_on='state_abbr', right_on='State')

In [ ]:
merged.plot()

In [ ]:
merged.to_file('./fig/shapefile/results/result_better_shape.shp', driver="ESRI Shapefile")